In [1]:
secret_scope = "scope"                                         # your secret scope
project = "key"                                                # the key in your secret scope that hold the CDF api-key you want to use
source_system = "orca"                                         # an identifier for the source system of the data
file_name = "Assets_v0.768"                                    # name for your file, we use this is name of the table in raw
file_location = "/FileStore/tables/wherever_it_is.xls"         # location of your file in dbfs
sheetName = "Sheet1"                                           # Excel "Sheet Name"
key_column = "id"                                              # name of the column in the sheet that we can use as unique identifier

In [2]:
new_table = spark.read.format("com.crealytics.spark.excel") \
    .option("location", file_location) \
    .option("useHeader", "true") \
    .option("treatEmptyValuesAsNulls", "true") \
    .option("inferSchema", "true") \
    .option("addColorColumns", "False") \
    .option("sheetName", sheetName) \
    .load()
new_table.createOrReplaceTempView("new_table")

In [3]:
# if base setup is already there
from cognite.client import CogniteClient

client = CogniteClient(api_key=dbutils.secrets.get(secret_scope, project))
db = source_system + "_dump"
table = file_name
client.raw.create_tables(database_name=db, table_names=[table])

In [4]:
# if raw needs to be set up
from cognite.client import CogniteClient

client = CogniteClient(api_key=dbutils.secrets.get(secret_scope, project))
db = "raw"
table = file_name
client.raw.create_databases([db])
client.raw.create_tables(database_name=db, table_names=[table])

db = "staging"
table = "asset_hierarchy"
client.raw.create_databases([db])
client.raw.create_tables(database_name=db, table_names=[table])

In [5]:
from pyspark.sql.functions import *
new_table = new_table.withColumn("key",col(key_column))

In [6]:
new_table_raw= spark.read.format("com.cognite.spark.datasource") \
    .option("type", "raw") \
    .option("database", "raw") \
    .option("table", file_name) \
    .option("apiKey", dbutils.secrets.get(secret_scope, project)) \
    .schema(new_table.schema) \
    .load()
new_table_raw.createOrReplaceTempView("new_table_raw")

In [7]:
new_table.write.insertInto("new_table_raw")

In [8]:
display(new_table_raw)